# The SNOMED Graph
## Reading Format
The SNOMED Graph is designed as a Directed Acyclic Graph, with a single root node called the Root Concept, or `SNOMED CT Concept (SNOMED RT+CTV3) |138875005|`. Each concept in SNOMED is connected to one or more concepts through various types of relationships, such as `|Is a|`, `|Part of|`, `|Has AMP|` etc. as you'll see later on in this notebook. Due to its size, importance, and the computational difficulty of generating it, the `|Is a|` relationship subgraph (list of all `Is a` relationship pairs of the form `A |is a| B`) is denormalized to a format called the [`transitive closure`](https://en.wikipedia.org/wiki/Transitive_closure#In_graph_theory) of the `|Is a|` relationship subgraph. For example, if we have this relationship structure:

+ ```A |is a| B```
+ ```B |is a| C```
+ ```C |is a| D```

Then the transitive closure will look like this: 

*`[(A,B),(A,C),(A,D),(B,C),(B,D),(C,D)]`*.

This data structure easily answers the question: `can I reach D from A by asking whether I can reach B from A, then asking whether I can reach C from B?`. If there's a tuple `(A,D)`, then the answer is yes. If the tuple doesn't exist, then the answer is No.

This example graph, with three simple relationships, spawns a transitive closure with six connections. The SNOMED transitive closure has 15,592,960 connections between 903,389 concepts.

In order to do anything relevant with SNOMED, such as collecting a list of drugs in the DM+D hierarchy (see below), we need this structure because it enables quick and efficient discovery of the local 'friends' of any given concept.

To make it easier to download this data structure, we precomputed this transitive closure as an [adjacency list (click here if you don't know what that is)](networkx.readthedocs.io/en/latest/reference/readwrite.adjlist.html)  using the parent-child relationship, and stored it in a file that can be easily downloaded from the terminology server API. [This link points to the file](http://snomedct-terminology-server-2016-07-04.slade360emr.com/terminology/relationships/transitive_closure/adjacency_list), and will automatically start a download if you click on it. 

In the two cells below, we've downloaded the adjacency list using `requests`, and saved it to a file 'children_subsumption.adjlist'. The file extension `.adjlist` is just plain text, so there's no extra processing required.

In [ ]:
import requests
response=requests.get(
   "http://snomedct-terminology-server-2016-07-04.slade360emr.com/terminology/relationships/transitive_closure/adjacency_list"
)

In [ ]:
with open('transitive_closure_adjacency_list.adjlist', 'w') as f:
   f.write(response.text)

## Start here if you don't want to download the adjacency list again.

In [2]:
import re
import requests
from itertools import chain
import networkx as nx
from simplejson import loads, load,dumps,dump
from requests.packages.urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

In [3]:
G=nx.DiGraph()
DG = nx.read_adjlist('transitive_closure_adjacency_list.adjlist', create_using=G)

In [4]:
DG.number_of_nodes()

904333

In [ ]:
DG.number_of_edges()

### Child concepts of 'Actual medicinal product'. These are the Actual Medicinal Products in our drug hierarchy

In [5]:
amp_successors = DG.successors('10363901000001102')

In [ ]:
len(amp_successors)

### DM+D Hierarchy identifiers - SCTIDs of AMP,AMPP,VMP,VTM,VMPP

In [6]:
amp=10363901000001102

In [7]:
vmpp=8653601000001108

In [8]:
vmp=10363801000001108

In [9]:
vtm=10363701000001104

In [10]:
# 10364001000001104
ampp=10364001000001104

In [11]:
dmd_hierarchy_identifiers = ['10363901000001102', '8653601000001108', '10363801000001108', '10363701000001104', '10364001000001104']

### Hierarchy identifiers for dm+d used in the creation of Dimension 1 (amp,vmp,vtm)

In [12]:
dimension_1_dmd_hierarchy_identifiers = ['10363901000001102', '10363801000001108', '10363701000001104']

### Obtain the SCTIDs for the children of "9191801000001103 |Trade family (product)|", which aren't needed here

In [13]:
trade_family = DG.successors('9191801000001103')

In [ ]:
#### example using zinnat: 228011000001102
#### get the 3 direct parents of "228011000001102 |Zinnat 125mg tablets (GlaxoSmithKline) (product)|"

In [ ]:
zinnat_predecessors=DG.predecessors('228011000001102')

In [ ]:
non_trade_zinnat_predecessors = [x for x in zinnat_predecessors if x not in trade_family]

In [ ]:
non_dmd_predecessors_of_zinnat = [x for x in non_trade_zinnat_predecessors if x not in dimension_1_dmd_hierarchy_identifiers]

In [ ]:
# returns the only parent of zinnat in the drug hierarchy
non_dmd_predecessors_of_zinnat

#### The amp-vmp map is correct once we remove trade family hierarchy and dmd hierarchy identifiers from the predecessors list

In [14]:
amp_drugs_and_their_vmp_parents = []
for amp_drug in amp_successors:
    parents_of_amp = DG.predecessors(amp_drug)
    for parent_of_amp in parents_of_amp:
        if not  parent_of_amp in trade_family and not parent_of_amp in dmd_hierarchy_identifiers:
            amp_drugs_and_their_vmp_parents.append({'amp': amp_drug, 'corresponding_vmp': parent_of_amp}) 

In [ ]:
print(len(amp_drugs_and_their_vmp_parents))
print(amp_drugs_and_their_vmp_parents[0])

In [ ]:
nx.ancestors(DG, '324506000')

In [15]:
pharma_sctid=373873005

In [16]:
# Get the children of "Virtual Medicinal Product"
vmp_successors = DG.successors(str(vmp))
vmp_successors[0]
# get the children of "Virtual Therapeutic Moiety"
vtm_successors = DG.successors(str(vtm))
# get the children of "Pharmaceutical/Biologic Product"
pharma_successors = DG.successors(str(pharma_sctid))
pharma_descendants = nx.descendants(DG, str(pharma_sctid))

In [17]:
vtms = []
for amp_vmp_map in amp_drugs_and_their_vmp_parents:
    # parents_of_vmp = DG.predecessors(amp_vmp_map['corresponding_vmp'])
    ancestors_of_vmp = nx.ancestors(DG, amp_vmp_map['corresponding_vmp'])
    for ancestor in ancestors_of_vmp:
        if ancestor in vtm_successors and not ancestor in dmd_hierarchy_identifiers:
            vtms.append({'amp': amp_vmp_map['amp'], 'vmp': amp_vmp_map['corresponding_vmp'], 'vtm': ancestor})

In [ ]:
print(len(vtms))

In [18]:
full_dmd = []

for amp_vmp_vtm_map in vtms:
    ancestors_of_vtm = nx.ancestors(DG, amp_vmp_vtm_map['vtm'])
    for ancestor in ancestors_of_vtm:
        if ancestor in pharma_successors:
            full_dmd.append({'amp': amp_vmp_vtm_map['amp'], 
                             'vmp': amp_vmp_vtm_map['vmp'], 
                             'vtm': amp_vmp_vtm_map['vtm'], 
                             'drug_class': ancestor})

In [ ]:
full_dmd_with_class_hops = []

for amp_vmp_vtm_map in vtms[:]:
    ancestors_of_vtm = nx.ancestors(DG, amp_vmp_vtm_map['vtm'])
    non_dmd_vtm_ancestors = set(ancestors_of_vtm).intersection(pharma_descendants)
    print()
    for ancestor in ancestors_of_vtm:
        if ancestor not in pharma_successors and not in 
        if ancestor in pharma_successors:
            full_dmd_with_class_hops.append({'amp': amp_vmp_vtm_map['amp'], 
                             'vmp': amp_vmp_vtm_map['vmp'], 
                             'vtm': amp_vmp_vtm_map['vtm'], 
                             'drug_class': ancestor})

In [ ]:
print(full_dmd[0])
print([key for key in full_dmd[0].keys()])
print([int(val) for val in full_dmd[0].values()])

In [ ]:
with open('dmd.json') as dmd_file:
    dmd_name_data = load(dmd_file)

In [20]:
concept_by_id_list_api = "http://snomedct-terminology-server-2016-07-04.slade360emr.com/terminology/concept_list_by_id/"
headers = {'Expect': '100-Continue'}
def get_concept_term_by_id_list(id_list):
    ids = ','.join(id_list)
    try:
        response = requests.post(concept_by_id_list_api,data={'sctid_list': ids}).text
    except Exception as e:
        print("Request failed at id {}. Retrying...".format(id_list))
        return get_concept_term_by_id_list(id_list)
    try:
        data = loads(response)
    except Exception as e:
        print(response)
    return data

get_concept_term_by_id_list([val for val in full_dmd[0].values()])

{'10363601000001109': 'UK product (product)',
 '319734006': 'Warfarin sodium 3mg tablet (product)',
 '48603004': 'Warfarin (product)',
 '508311000001104': 'Warfarin 3mg tablets (Teva UK Ltd)'}

In [ ]:
# extract all of the ids from the dict, into a list, 
# then get their preferred terms from the SNOMED CT Termserver API
dmd_values = [list(obj.values()) for obj in full_dmd]
dmd_id_set=set(list(chain(chain(*dmd_values))))
print("number of sctids in this hierarchy: {}".format(len(dmd_id_set)))
dmd_name_data=get_concept_term_by_id_list(list(dmd_id_set))

In [ ]:
dmd_data_flattened = []
for dmd_datum in full_dmd:
    dmd_data_flattened.append(
    {'drug_class': dmd_name_data[dmd_datum['drug_class']] + ' | {}'.format(dmd_datum['drug_class']),
     'amp' : dmd_name_data[dmd_datum['amp']]  + ' | {}'.format(dmd_datum['amp']),
     'vmp': dmd_name_data[dmd_datum['vmp']] + ' | {}'.format(dmd_datum['vmp']),
     'vtm': dmd_name_data[dmd_datum['vtm']] + ' | {}'.format(dmd_datum['vtm'])
     })

In [ ]:
# check consistency of the flattened dm+d data, 
# by confirming that the sctid concatenated with the pipe is correct for the preferred term,
# for any given row
dmd_data_flattened[0]

In [ ]:
# i save the dmd data to a file in case anything goes wrong (i.e. i write a destructive function that modifies this data)
with open('dmd_data_flattened.json', 'w') as f:
    dump(dmd_data_flattened, f)
    

In [ ]:
columns_map = map(lambda x: x.keys(), dmd_data_flattened)
columns_list = chain.from_iterable([list(list(column)) for column in columns_map])
columns=list(set(list(columns_list)))
columns = ['amp', 'vmp', 'vtm', 'drug_class']

In [ ]:
# Confirm whether a given row has the correct structure. 
# If the results of line 2 match (by column) those of line 1, we're good
print(columns)
list(map(lambda x: dmd_data_flattened[0].get(x,""),columns))

In [ ]:
# write data to csv file
import csv
with open('dmd_data_full.csv', 'w') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(columns)
    for input_row in dmd_data_flattened:
        csv_writer.writerow(list(map(lambda x: input_row.get(x,""),columns)))

# Teaching Case: DM+D dimensional table that includes AMPP and VMPP
+ This will be tricker than the above case, due to dm+d's hierarchy. See the image:

![dm+d hierarchy](dmd_hierarchy_with_reltypes.png "DM+D Hierarchy With relationship type")

### The predecessors of AMPP (Actual Medicinal Product Pack)

In [21]:
# relationship type id for the "Has VMP" relationship type, whose destination id is always a vmp successor.
has_vmp_rel_type_id=10362601000001103

In [22]:
# relationship type id for the "Has AMP" relationship type, whose destination id is always an amp successor.
has_amp_rel_type_id=10362701000001108

In [23]:
destination_ids_api = "http://snomedct-terminology-server-2016-07-04.slade360emr.com/terminology/relationships/destination_by_type_id/{}"
def get_destination_id_by_reltype(type_id):
    print(type_id)
    try:
        response = requests.get(destination_ids_api.format(type_id)).text
    except Exception as e:
        print("Request failed. Retrying...".format(type_id))
        print(response)
    try:
        data = loads(response)
    except Exception as e:
        print(response)
    return data

In [24]:
# ampp successors are drugs under the 'AMPP (Actual medicinal product pack)' sub-hierarchy of DM+D
ampp_successors = DG.successors(str(ampp))

In [25]:
'1329211000001109' in ampp_successors

True

In [26]:
# This returns a large dict, with the format
# {'ampp_drug_code_1':'amp_drug_code_1','ampp_drug_code_2':'amp_drug_code_2',...}
# The ampp drug points to its amp parent.
ampp_amp_map = get_destination_id_by_reltype(has_amp_rel_type_id)

10362701000001108


In [27]:
non_dmd_predecessors_of_ampp_drugs = []
for ampp in ampp_successors:
    non_dmd_predecessors_of_ampp_drugs.append(
        {
            'ampp': ampp,
            'amp': ampp_amp_map[ampp]
        }
    )

In [28]:
non_dmd_predecessors_of_ampp_drugs[0]

{'amp': '17787911000001101', 'ampp': '17788111000001103'}

### The VMP predecessors of vmpp

In [29]:
# vmpp successors are drugs under the 'VMPP (Virtual medicinal product pack)' sub-hierarchy of DM+D
print(vmpp)
vmpp_successors = DG.successors(str(vmpp))

8653601000001108


In [30]:
# This returns a large dict, with the format
# {'vmpp_drug_code_1':'vmp_drug_code_1','vmpp_drug_code_2':'vmp_drug_code_2',...}
# The vmpp drug points to its vmp parent.
vmpp_vmp_map = get_destination_id_by_reltype(has_vmp_rel_type_id)

10362601000001103


In [31]:
vmpp_successors[0]

'4715911000001102'

In [ ]:
len(vmpp_vmp_map.keys())

In [32]:
list(vmpp_vmp_map.keys())[0] in vmpp_successors

True

In [33]:
vmpp_vmp_map['10235211000001109'] == '10246111000001108'

True

In [34]:
non_dmd_predecessors_of_vmpp = []
for vmpp in vmpp_successors:
    non_dmd_predecessors_of_vmpp.append(
    {
            'vmpp': vmpp,
            'vmp': vmpp_vmp_map[vmpp]
    }
    )

In [ ]:
non_dmd_predecessors_of_vmpp[0]

In [35]:
ampp_drugs_with_vtms = []
for ampp in non_dmd_predecessors_of_ampp_drugs:
    for vtm in vtms:
        if vtm['amp'] == ampp['amp']:
            ampp_drugs_with_vtms.append(
                {'amp': ampp['amp'], 
                 'ampp': ampp['ampp'], 
                 'vmp': vtm['vmp'], 
                 'vtm': vtm['vtm']
                }
            )

In [36]:
ampp_drugs_with_vtms[10]

{'amp': '23599111000001103',
 'ampp': '23599211000001109',
 'vmp': '318672001',
 'vtm': '84078002'}

In [37]:
vmpp_drugs_with_ampps = []
for vmpp in non_dmd_predecessors_of_vmpp:
    for ampp in ampp_drugs_with_vtms:
        if ampp['vmp'] == vmpp['vmp']:
            vmpp_drugs_with_ampps.append({'amp': ampp['amp'], 'ampp': ampp['ampp'], 'vtm': ampp['vtm'], 'vmp': vmpp['vmp'], 'vmpp': vmpp['vmpp']})

In [38]:
vmpp_drugs_with_ampps[0]

{'amp': '4716611000001103',
 'ampp': '4716711000001107',
 'vmp': '334216004',
 'vmpp': '4715911000001102',
 'vtm': '82573000'}

In [41]:
full_vmpp_dmd_with_drug_class = []

for ampp_vmpp_amp_vmp_vtm_map in vmpp_drugs_with_ampps:
    ancestors_of_vtm = nx.ancestors(DG, ampp_vmpp_amp_vmp_vtm_map['vtm'])
    for drug_class in ancestors_of_vtm:
        dmd_row = {}
        dmd_row.update({'amp': ampp_vmpp_amp_vmp_vtm_map['amp'],
                        'ampp': ampp_vmpp_amp_vmp_vtm_map['ampp'],
                        'vmp': ampp_vmpp_amp_vmp_vtm_map['vmp'],
                        'vmpp': ampp_vmpp_amp_vmp_vtm_map['vmpp'],
                        'vtm': ampp_vmpp_amp_vmp_vtm_map['vtm']})
        if drug_class in pharma_successors:
            dmd_row.update({'drug_class': drug_class})
            full_vmpp_dmd_with_drug_class.append(dmd_row)

In [121]:
full_vmpp_dmd_with_drug_class[0]

{'amp': '4716611000001103',
 'ampp': '4716711000001107',
 'drug_class': '10363601000001109',
 'vmp': '334216004',
 'vmpp': '4715911000001102',
 'vtm': '82573000'}

In [48]:
drug_class_descendants = {x: nx.descendants(DG, x) for x in set(map(lambda x: x.get('drug_class'), full_vmpp_dmd_with_drug_class))}

In [66]:
full_dmd_with_all_class_hops = []
for row in full_vmpp_dmd_with_drug_class:
    dmd_row = {}
    dmd_row.update(row)
    ancestors_of_vtm = nx.ancestors(DG, row['vtm'])
    descendants_of_drug_class = drug_class_descendants[row['drug_class']]
    drug_class_vtm_hops = set(descendants_of_drug_class).intersection(ancestors_of_vtm)
    class_hops = list(set(drug_class_vtm_hops).difference(dmd_hierarchy_identifiers))
    class_hops_dict = {'class_{}'.format(class_hops.index(drug) + 1): drug for drug in class_hops}
    dmd_row.update(class_hops_dict)
    full_dmd_with_all_class_hops.append(dmd_row)

In [71]:
# extract all of the ids from the dict, into a list, 
# then get their preferred terms from the SNOMED CT Termserver API
vmpp_dmd_values = [list(obj.values()) for obj in full_dmd_with_all_class_hops]
vmpp_dmd_id_set=set(list(chain(chain(*vmpp_dmd_values))))
print("number of sctids in this hierarchy: {}".format(len(vmpp_dmd_id_set)))
vmpp_dmd_name_data=get_concept_term_by_id_list(list(vmpp_dmd_id_set))

number of sctids in this hierarchy: 131882


#### Regexp test cases. In order to extract company names, we'll need to parse all of these strings and get (only) the company name.
If the regex (or function) created works on all these strings, then the company name has been correctly extracted.

In [119]:
string_1 = "Ethanol 100% solution for injection 2ml ampoules (Alliance Healthcare (Distribution) Ltd)"
string_2 = "Manusept antibacterial hand rub (Medlock Medical) (product)"
string_3 = "Rapiscan 400micrograms/5ml solution for injection vials (Rapidscan Pharma Solutions EU Ltd) (product)"
string_4 = 'Co-codamol 8mg/500mg tablets (Almus Pharmaceuticals Ltd)'
string_5 = 'Acular 0.5% eye drops (PI) (Waymade Ltd) (product)'
string_6 = 'Minijet lidocaine 100mg/10ml (1%) solution for injection pre-filled syringes (Celltech Pharmaceuticals Ltd) (product)'
terms = [string_1, string_2, string_3, string_4, string_5, string_6]

In [117]:
company_name_expression=r'(\(([^)]+)\)([\s\w+\)]+|))'
def extract_company_name_from_term(term):
    term_with_pi_extracted = term.split('(PI)')
    term_length = len(term_with_pi_extracted)
    if term_length > 1:
        match = re.search(company_name_expression, term_with_pi_extracted[1])
        try:
            return match.group(0).strip()[1:-1]
        except AttributeError:
            print(">>>> {}".format(term_with_pi_extracted))                                                             
    elif term_length == 1:
        percentage_values = re.split(r'\(\d{1,3}%\)', term_with_pi_extracted[0])
        if len(percentage_values) > 1:
            match = re.search(company_name_expression, percentage_values[1])
            try:
                return match.group(0).strip()[1:-1]
            except AttributeError:
                print("$$$$$ {}".format(percentage_values))
        elif len(percentage_values) == 1:
            match = re.search(company_name_expression, percentage_values[0])
            try:
                return match.group(0).strip()[1:-1]
            except AttributeError:
                print("++++ {}".format(percentage_values))

In [137]:
list(map(lambda x: '{}: {}'.format(extract_company_name_from_term(x), x), terms))

['Alliance Healthcare (Distribution) Ltd: Ethanol 100% solution for injection 2ml ampoules (Alliance Healthcare (Distribution) Ltd)',
 'Medlock Medical: Manusept antibacterial hand rub (Medlock Medical) (product)',
 'Rapidscan Pharma Solutions EU Ltd: Rapiscan 400micrograms/5ml solution for injection vials (Rapidscan Pharma Solutions EU Ltd) (product)',
 'Almus Pharmaceuticals Ltd: Co-codamol 8mg/500mg tablets (Almus Pharmaceuticals Ltd)',
 'Waymade Ltd: Acular 0.5% eye drops (PI) (Waymade Ltd) (product)',
 'Celltech Pharmaceuticals Ltd: Minijet lidocaine 100mg/10ml (1%) solution for injection pre-filled syringes (Celltech Pharmaceuticals Ltd) (product)']

In [105]:
vmpp_dmd_data_flattened = []
for dmd_datum in full_dmd_with_all_class_hops:
    row = {}
    row.update({key: 
                vmpp_dmd_name_data[dmd_datum[key]] + ' | {}'.format(dmd_datum[key]) 
                for key in dmd_datum.keys()})

    row.update({'company_name': extract_company_name_from_term(row['amp'])})
    vmpp_dmd_data_flattened.append(row)

In [132]:
vmpp_dmd_data_flattened[1000]

{'amp': 'Paracetamol 500mg / Dihydrocodeine 20mg tablets (Alliance Healthcare (Distribution) Ltd) (product) | 20363711000001101',
 'ampp': 'Paracetamol 500mg / Dihydrocodeine 20mg tablets (Alliance Healthcare (Distribution) Ltd) 56 tablet | 20363811000001109',
 'company_name': 'Alliance Healthcare (Distribution) Ltd',
 'drug_class': 'UK product (product) | 10363601000001109',
 'vmp': 'Acetaminophen + dihydrocodeine 500mg/20mg tablet (product) | 322367008',
 'vmpp': 'Paracetamol 500mg / Dihydrocodeine 20mg tablets 56 tablet (product) | 1211411000001108',
 'vtm': 'Paracetamol | 90332006'}

In [107]:
columns_map = map(lambda x: x.keys(), vmpp_dmd_data_flattened)
columns_list = chain.from_iterable([list(list(column)) for column in columns_map])
columns=list(set(list(columns_list)))

# write data to csv file
import csv
with open('dmd_data_ampp_level_with_company_names_and_class_hops.csv', 'w') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(columns)
    for input_row in vmpp_dmd_data_flattened:
        csv_writer.writerow(list(map(lambda x: input_row.get(x,""),columns)))

### Has dose form

In [ ]:
has_dose_form_relationship_type_id=411116001

In [ ]:
has_dose_form_destination_ids=get_destination_id_by_reltype(has_dose_form_relationship_type_id)

In [ ]:
set(map(lambda x: int(x), list(has_dose_form_destination_ids.keys())[1:100]))